# Otimização de Consultas no Spark


### Entendendo o plano de execução (Tipos de Operators e Regras)

---

**Operators:**

* FileScan
* Exchange
* HashAggregate, SortAggregate, ObjectHashAggregate
* SortMergeJoin
* BroadcastHashJoin

O operador FileScan representa a leitura de dados a partir de um formato qualquer de arquivo. Trabalha com 3 tipos de filtros, DataFilters, PushedFilters e PartitionFilters.


O operador Exchange realiza o shuffle (embaralhamento) e a movimentação física (transferência) dos dados no cluster, essa movimentação de dados entre os nós é bastante onerosa e de forma geral, deve ser evitada. O exchange é induzido pelas transformações e possui as seguintes estratégias de particionamento:

- Exchange HashPartitioning: 


    - groupBy
    - distinct
    - join
    - repartition("key")
    - Window.partitionBy("key")
    
    
- Exchange SinglePartition (todos os dados serão movidos para uma única partição):



    - Window.partitionBy()  
    
    
- Exchange RoundRobinPartitioning (dados são movidos para um número estabelecido de partições):



    - repartition(10)  
    
    
- Exchange RangePartitioning:



    - orderBy
    
    
    

O operador Aggregate representa uma agregação nos dados, é induzido pelas transformações groupBy, distinct e DropDuplicates e possui as seguintes estratégias (que são atribuídas em runtime).

- HashAggregate     
- SortAggregate
- ObjectHashAggregate
    

O operador SortMergeJoin representa a junção entre dois dataframes e a ideia geral desse operador é primeiro ordenar os dados pelos valores contidos nas colunas que realizam o join para que as varreduras lineares intercaladas encontrem esses conjuntos ao mesmo tempo e, em seguida, realiza a movimentação dos dados para que cada executor mantenha uma parte específica desses dados. Como você pode imaginar, esse tipo de estratégia é onerosa: os nós precisam usar a rede para compartilhar dados e dependendo do volume, isso pode se tornar inviável.


O operador BroadcastHashJoin também representa a junção entre dois dataframes porém utilizando outra abordagem. Durante a execução, ele se sub-divide em dois jobs com funções distintas:

- BroadcastExchange
- BroadcastHashJoin

No BroadcastExchange o Spark envia uma cópia inteira de uma lookup table para cada executor, desta forma cada executor é autossuficiente na execução de operações de join. 


---

**Rules:**

- EnsureRequirements
- ReuseExchange



### Adaptive Query Execution (AQE)

Adaptive Query Execution (AQE) é um dos melhores recursos do Spark 3.0, que reotimiza e ajusta os planos de consulta com base nas estatísticas de tempo de execução coletadas durante a execução da consulta. 

Depois de habilitar a AQE, as seguintes melhorias serão realizadas:

 - Conversão automatica do sort-merge join (lento) para o Broadcast join
 - Otimização do Skew Join (dados distribuidos de forma desigual entre as partições no cluster)
 - Coalescing Post-shuffle Partitions que determinam dinamicamente o número ideal de partições


para habilitaro AQS: 

* spark.conf.set("spark.sql.adaptive.enabled", "true")


In [2]:
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F

spark = SparkSession\
    .builder\
    .appName("Spark Performance Issues")\
    .config("spark.sql.adaptive.enabled", "false")\
    .config("spark.sql.parser.ansi.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.metrics.conf.*.sink.console.class", "org.apache.spark.metrics.sink.ConsoleSink")\
    .getOrCreate()

sql_context = SQLContext(sc)

### Simulando um desbalanceamento entre as partições (Data Skew)


Data Skew é uma condição em que os dados de uma tabela são distribuídos de forma desigual entre as partições no cluster. A distorção de dados pode prejudicar gravemente o desempenho das consultas, especialmente aquelas com junções. As junções entre tabelas grandes exigem dados embaralhados (shuffling) e a distorção pode levar a um  desequilíbrio extremo de trabalho no cluster. É provável que a distorção de dados esteja afetando a performance de uma consulta caso ela pareça estar travada ao termino de poucas tasks (por exemplo, as últimas 3 tasks de 200). 

O Spark foi desenvolvido para funcionar com partições de tamanhos iguais e o objetivo é espalhar as partições para os executores da maneira mais uniforme possível. O particionamento de hash tenta espalhar os dados da maneira mais uniforme possível em todas as partições com base nas chaves de join.



In [24]:
spark.conf.set("spark.sql.shuffle.partitions", 10)

spark.range(1000000)\
    .withColumn("join_key", F.lit(" "))\
    .write\
    .mode("overwrite")\
    .saveAsTable("table_x")

spark.range(1000000)\
    .withColumn("join_key", F.lit(" "))\
    .write\
    .mode("overwrite")\
    .saveAsTable("table_y")


No código a seguir, será realizado o join entre as duas tabelas, o que irá produzir um output de um trilhão de linhas, e todas elas serão produzidas utilizando um único executor (o executor que obtém a chave ""):

Nesse processo, quando a action for executado o Spark ira computar o hash da coluna do join e realiza uma ordenação. Em seguida, ele tenta manter os registros com os mesmos hashes de ambas as partições no mesmo executor de forma que todos os valores nulos da tabela vão para um executor e o spark entra em um loop contínuo de embaralhamento e coleta de lixo, até falhar. Nesse caso, há apenas uma chave de join problemática. Em outros cenários, podem existir outros fatores.


In [25]:
query = """
    SELECT id, count()
    FROM
    (
        SELECT x.id
        FROM table_x x
        JOIN table_y y ON x.join_key = y.join_key
    )
    GROUP BY id
"""

df = sql_context.sql(query)

In [26]:
df.explain(extended = True)

== Parsed Logical Plan ==
'Aggregate ['id], ['id, unresolvedalias('count(), None)]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['x.id]
      +- 'Join Inner, ('x.join_key = 'y.join_key)
         :- 'SubqueryAlias x
         :  +- 'UnresolvedRelation [table_x]
         +- 'SubqueryAlias y
            +- 'UnresolvedRelation [table_y]

== Analyzed Logical Plan ==
id: bigint, count(): bigint
Aggregate [id#17L], [id#17L, count() AS count()#372L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [id#17L]
      +- Join Inner, (join_key#19 = join_key#338)
         :- SubqueryAlias x
         :  +- SubqueryAlias table_x
         :     +- Project [id#17L,   AS join_key#19]
         :        +- Range (0, 1000000, step=1, splits=Some(4))
         +- SubqueryAlias y
            +- SubqueryAlias table_y
               +- Project [id#336L,   AS join_key#338]
                  +- Range (0, 1000000, step=1, splits=Some(4))

== Optimized Logical Plan ==
Aggregate [id#17L],

In [ ]:
# se a action for disparada o job entrará em loop infinito!!!
df.show()

Vamos testar a nova funcionalidade do spark que promete aumentar o desempenho nessas situações

In [8]:
spark.conf.set("spark.sql.adaptive.enabled", "true")

In [ ]:
# disparando a action com a config habilitada (mesmo com a otimização habilitado, entrou em loop infinito)
df.show()

In [ ]:
df.explain()

### Tentativa de otimização utilizando uma outra abordagem:

Vamos dividir a tabela em duas partes. A primeira parte conterá todas as linhas que não têm uma chave nula e a segunda parte conterá todos os dados sem valores nulos.

Em seguida, iremos alterar / reescrever nossa lógica ETL, onde podemos realizar um join à esquerda com a tabela not_null e simplesmente executar uma união com a coluna nula, pois as chaves nulas não participarão do join. Portanto, ao seguir essa técnica, podemos evitar um embaralhamento e o problema de pausa GC na tabela com grandes valores nulos.



In [18]:
sql_context.sql("SELECT * FROM table_x WHERE join_key IS NULL").createOrReplaceTempView("table_x_id_null")
sql_context.sql("SELECT * FROM table_x WHERE join_key IS NOT NULL").createOrReplaceTempView("table_x_id_not_null")

sql_context.sql("SELECT * FROM table_y WHERE join_key IS NULL").createOrReplaceTempView("table_y_id_null")
sql_context.sql("SELECT * FROM table_y WHERE join_key IS NOT NULL").createOrReplaceTempView("table_y_id_not_null")

# reescrever a query

df = sql_context.sql("select COUNT(0), join_key from table_y GROUP BY join_key")
df.show()


+--------+--------+
|count(0)|join_key|
+--------+--------+
| 1000000|        |
+--------+--------+



In [ ]:
df.rdd.getNumPartitions()

In [ ]:
# analisa a distribuição dos dados nas das partições
for i, part in enumerate(df.rdd.glom().collect()):
    print({i: part})
